In [86]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime
import time
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from math import floor, ceil
from pylab import rcParams
import re

%matplotlib inline

In [87]:
engine = sqlite3.connect('DB')

storesWithTrain = pd.read_sql("""
SELECT *
FROM trainDf
JOIN storesDf 
ON storesDf.Store = trainDf.Store
""", engine)

_, i = np.unique(storesWithTrain.columns, return_index = True)
storesWithTrain = storesWithTrain.iloc[:, i]
storesWithTrain.to_sql('storesWithTrain', engine, if_exists='replace', index = False)

everything = pd.read_sql("""
SELECT *
FROM storesWithTrain
JOIN featuresDf
ON storesWithTrain.Store = featuresDf.Store AND storesWithTrain.Date = featuresDf.Date
""", engine)

_, i = np.unique(everything.columns, return_index = True)
everything = everything.iloc[:, i]

In [88]:
train_x = everything.drop_duplicates()
#Convert date string to date time object to get milliseconds
train_x['Date'] = train_x['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
train_x['Date'] = train_x['Date'].apply(lambda x: int(time.mktime(x.timetuple())) / 100000)
train_y = train_x['Weekly_Sales']
train_y = train_y.to_frame()
train_x.drop('Weekly_Sales', axis = 1, inplace = True)

In [89]:
#Optional cell if you want to remove NaNs. This will mainly remove the MarkDown rows
#Also changed the store types from A and B to 1 and 2. Or could one hot encode
storeType = train_x['Type']
encoding = pd.get_dummies(storeType)
#encoding = encoding.drop('C', axis = 1)

train_x['A'] = encoding['A']
train_x['B'] = encoding['B']
train_x['C'] = encoding['C']
train_x.drop('Type', axis = 1, inplace = True)
train_x = train_x.dropna().reset_index(drop = True)

In [90]:
train_x

,CPI,Date,Dept,Fuel_Price,IsHoliday,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,Size,Store,Temperature,Unemployment,A,B,C
0,217.998085,13209.876,1,3.297,0,10382.90,6115.67,215.07,2406.62,6551.42,151315,1,59.11,7.866,1,0,0
1,218.220509,13215.924,1,3.308,0,6074.12,254.39,51.98,427.39,5988.57,151315,1,62.25,7.866,1,0,0
2,218.467621,13221.972,1,3.236,1,410.31,98.00,55805.51,8.00,554.92,151315,1,60.14,7.866,1,0,0
3,218.714733,13228.020,1,3.172,0,5629.51,68.00,1398.11,2084.64,20475.32,151315,1,48.91,7.866,1,0,0
4,218.961846,13234.068,1,3.158,0,4640.65,19.00,105.02,3639.42,14461.82,151315,1,43.93,7.866,1,0,0
5,219.179453,13240.116,1,3.159,0,5011.32,67.00,347.37,225.79,4011.37,151315,1,51.63,7.866,1,0,0
6,219.357722,13246.164,1,3.112,0,2725.36,40.48,634.70,24.90,2739.43,151315,1,47.96,7.866,1,0,0
7,219.535990,13252.212,1,3.129,1,5762.10,46011.38,260.36,983.65,4735.78,151315,1,44.55,7.866,1,0,0
8,219.714258,13258.260,1,3.157,0,6277.39,21813.16,143.10,1450.13,8483.00,151315,1,49.01,7.348,1,0,0
9,219.892526,13264.308,1,3.261,0,5183.29,8025.87,42.24,453.08,3719.38,151315,1,48.53,7.348,1,0,0


In [91]:
train_x.to_sql('train_x', engine, if_exists = 'replace', index = False)
train_y.to_sql('train_y', engine, if_exists = 'replace', index = False)